# Fine-Tuning a Conversational LLM for Psychological Dialogue Support

This project fine-tunes a Large Language Model (LLM) to simulate natural, empathetic, and context-aware conversations between a psychologist and a patient. The model learns from a curated dataset of real or synthetic therapy-style question–answer exchanges, allowing it to generate emotionally intelligent, coherent, and supportive responses to mental-health-related queries.

Unlike general chatbots trained on open-domain text, this model specializes in therapeutic conversation patterns — focusing on reflective listening, validating emotions, and suggesting healthy thought reframing.

The final output is an AI-driven conversational agent that can engage in mental-wellness dialogue, provide psychoeducation, and guide users toward constructive self-reflection — without offering clinical diagnosis or treatment.

# Import Libraries

In [1]:
import pandas as pd

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 1000)

In [2]:
import torch
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Trainer, TrainingArguments
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments



# Import Dataset and Preprocess it

In [3]:
# Load and preprocess the data
df_fine_tuning = pd.read_csv('train.csv')
df_fine_tuning = df_fine_tuning.dropna()

df_fine_tuning = df_fine_tuning.dropna().apply(lambda x: x.str.strip())
df_fine_tuning = df_fine_tuning.sample(n=20, random_state=42)
# Shuffle and split the dataset
train_data = df_fine_tuning.sample(frac=0.8, random_state=42)
eval_data = df_fine_tuning.drop(train_data.index)





In [4]:
train_data.head(1)

,Context,Response
3155,We've been in a long distance relationship for two and a half years. I recently saw his phone and saw the people he texts the most and one of them was a female coworker. I don't know how to approach this situation. How do I ask him about it?.,"If you'd like to ask a question, then go ahead and ask!Boyfriend/girlfriend is a close relationship and it is usually understood as an exclusive relationship. You're definitely entitled to know if your wishes to not have him texting another woman, are being respected.Often people are afraid to ask because they fear the truth will hurt them.In the short term this is definitely true.In the long term, knowing you are getting what you want and at the very least stating your expectations to your boyfriend, will clarify for him, what is meaningful in your relationship."


## Download the Model and its tokenizer


In [5]:
tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium") # Load the tokenizer
model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium") # Load the model
# Set the eos_token as the pad_token
tokenizer.pad_token = tokenizer.eos_token # Set the pad_token to eos_token for proper padding 

#ignore warnings
import warnings
warnings.filterwarnings("ignore")

## Tokenize the dataset for training

In [ ]:
# Define the custom dataset class
class ChatDataset(Dataset):
    def __init__(self, dataframe, tokenizer, source_max_length=512, target_max_length=512):
        self.tokenizer = tokenizer
        self.data = dataframe
        self.source_max_length = source_max_length
        self.target_max_length = target_max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        row = self.data.iloc[index]
        source_text = row['Context']
        target_text = row['Response']

        # Tokenize and encode the source and target texts
        source_encoding = self.tokenizer(
            source_text,
            max_length=self.source_max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            target_text,
            max_length=self.target_max_length,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        labels = target_encoding['input_ids']
        labels[labels == self.tokenizer.pad_token_id] = -100  # Ignore padding in loss calculation

        return {
            'input_ids': source_encoding['input_ids'].flatten(),
            'attention_mask': source_encoding['attention_mask'].flatten(),
            'labels': labels.flatten()
        }

# Create datasets
train_dataset = ChatDataset(train_data, tokenizer)
eval_dataset = ChatDataset(eval_data, tokenizer)



train_dataloader = DataLoader(train_dataset, batch_size=20, shuffle=True)
eval_dataloader = DataLoader(eval_dataset, batch_size=20)


In [7]:
training_args = TrainingArguments(
    output_dir='./results',          # Output directory
    num_train_epochs=5,              # Total number of training epochs
    per_device_train_batch_size=10,   # Batch size per device during training
    per_device_eval_batch_size=10,    # Batch size for evaluation
    warmup_steps=4,                # Number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    report_to="none",
    logging_steps=10, ## use 1 for logging traning loss at every step
    save_total_limit=2,
    eval_strategy="epoch",
    save_strategy="epoch"
)

trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_dataset,         # Training dataset
    eval_dataset=eval_dataset            # Evaluation dataset
)

trainer.train()


`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,No log,12.595261
2,No log,10.693070
3,No log,8.845859
4,No log,7.951909
5,10.314600,7.431581


TrainOutput(global_step=10, training_loss=10.31457061767578, metrics={'train_runtime': 2982.7386, 'train_samples_per_second': 0.027, 'train_steps_per_second': 0.003, 'total_flos': 74296055562240.0, 'train_loss': 10.31457061767578, 'epoch': 5.0})

In [8]:
# Save the fine-tuned model
model.save_pretrained('./fine_tuned_model')
tokenizer.save_pretrained('./fine_tuned_model')


('./fine_tuned_model\\tokenizer_config.json',
 './fine_tuned_model\\special_tokens_map.json',
 './fine_tuned_model\\chat_template.jinja',
 './fine_tuned_model\\vocab.json',
 './fine_tuned_model\\merges.txt',
 './fine_tuned_model\\added_tokens.json',
 './fine_tuned_model\\tokenizer.json')